In [ ]:
import firebase_admin
from firebase_admin import credentials, firestore
from django.contrib.gis.geos import Point
from django.contrib.gis.db import models
from django.conf import settings
from django.core.exceptions import ObjectDoesNotExist
from django.http import JsonResponse
from django.views import View
import json
import uuid
from datetime import datetime
from django.utils import timezone
# Initialize Firebase Admin SDK (replace with your credentials)
if not firebase_admin._apps:
    cred = credentials.Certificate("path/to/your/firebase_credentials.json")  # Change this to your credentials file path
    firebase_admin.initialize_app(cred)

# Get Firestore client
db = firestore.client()


# MARK: - Django Models with GeoDjango
class User(models.Model):
    """
    Django model representing a user.  This is intended to store *additional*
    user data that we want in our own database, *not* authentication data.
    Authentication will be handled by Firebase.
    """
    user_id = models.CharField(max_length=255, unique=True)  # Firebase User ID
    name = models.CharField(max_length=255)
    email = models.EmailField()
    phone = models.CharField(max_length=20)
    address = models.CharField(max_length=255)

    def __str__(self):
        return self.name


class Delivery(models.Model):
    """
    Django model representing a delivery.
    """
    delivery_id = models.UUIDField(primary_key=True, default=uuid.uuid4, editable=False)
    pickup_location = models.CharField(max_length=255)  # Store as text, or use a PointField as below
    dropoff_location = models.CharField(max_length=255)
    status = models.CharField(max_length=20, default="pending")  # e.g., "pending", "in transit", "delivered"
    assigned_rider_id = models.CharField(max_length=255, null=True, blank=True)  # Firebase User ID of rider
    request_time = models.DateTimeField(default=timezone.now)
    pickup_time = models.DateTimeField(null=True, blank=True)
    dropoff_time = models.DateTimeField(null=True, blank=True)
    cost = models.DecimalField(max_digits=10, decimal_places=2)
    notes = models.TextField(null=True, blank=True)
    #location = models.PointField(geography=True, null=True, blank=True) #use this if you want to store the location as a point. Requires PostGIS

    def __str__(self):
        return f"Delivery {self.delivery_id} - {self.status}"


# MARK: - Route Optimization (Placeholder)
class RouteOptimizer:
    """
    Placeholder class for route optimization.
    """
    @staticmethod
    def optimize_route(deliveries):
        """
        Placeholder for a route optimization function.
        Args:
            deliveries (list[Delivery]): A list of Delivery objects.
        Returns:
            list[Delivery]: A list of Delivery objects, reordered in an optimized route.
        """
        # In a real app, this would involve a more complex algorithm (e.g.,
        # using a dedicated routing API like Google Maps, or a TSP solver).
        # For simplicity, just return the deliveries in the original order.
        return deliveries

# MARK: - Helper Functions
def get_user_from_firebase(firebase_user_id):
    """
    Fetches a user's data from Firebase Authentication, and creates a User object
    in our local database if it doesn't already exist.
    Args:
        firebase_user_id (str): The user's ID from Firebase Authentication.
    Returns:
        User: The User object from our local database, or None on error.
    """
    try:
        user = firebase_admin.auth.get_user(firebase_user_id)
        # Get or create the user in our local database
        local_user, created = User.objects.get_or_create(
            user_id=firebase_user_id,
            defaults={
                'name': user.display_name,
                'email': user.email,
            }
        )
        return local_user
    except firebase_admin.auth.UserNotFoundError:
        print(f"User not found in Firebase: {firebase_user_id}")
        return None
    except Exception as e:
        print(f"Error fetching user from Firebase: {e}")
        return None



# MARK: - Django Views
class DeliveryRequestView(View):
    """
    View to handle delivery requests.
    """
    def post(self, request):
        try:
            data = json.loads(request.body)
            #Get firebase user.
            firebase_user_id = data.get('userId') #Get from request.
            if not firebase_user_id:
                return JsonResponse({'error': 'User ID is required'}, status=400)
            user = get_user_from_firebase(firebase_user_id)
            if not user:
                 return JsonResponse({'error': 'User not found'}, status=404)

            pickup_location = data.get('pickupLocation')
            dropoff_location = data.get('dropoffLocation')
            cost = data.get('cost')
            notes = data.get('notes')

            if not all([pickup_location, dropoff_location, cost]):
                return JsonResponse({'error': 'Missing required fields'}, status=400)

            try:
                cost = float(cost)
            except ValueError:
                return JsonResponse({'error': 'Invalid cost value'}, status=400)

            new_delivery = Delivery(
                pickup_location=pickup_location,
                dropoff_location=dropoff_location,
                cost=cost,
                notes=notes,
            )
            new_delivery.save()

            # Store delivery in Firestore (for demonstration purposes)
            delivery_data = {
                'delivery_id': str(new_delivery.delivery_id),
                'pickup_location': pickup_location,
                'dropoff_location': dropoff_location,
                'status': new_delivery.status,
                'request_time': new_delivery.request_time,
                'cost': cost,
                'notes': notes,
                'user_id': user.user_id  # Store the Firebase User ID
            }
            db.collection('deliveries').document(str(new_delivery.delivery_id)).set(delivery_data)

            return JsonResponse({'message': 'Delivery request created successfully'}, status=201)
        except json.JSONDecodeError:
            return JsonResponse({'error': 'Invalid JSON'}, status=400)
        except Exception as e:
            return JsonResponse({'error': str(e)}, status=500)



class DeliveryListView(View):
    """
    View to retrieve a list of deliveries.
    """
    def get(self, request):
        deliveries = Delivery.objects.all().order_by('-request_time')  # Order by request time
        deliveries_data = [{
            'delivery_id': str(delivery.delivery_id),
            'pickup_location': delivery.pickup_location,
            'dropoff_location': delivery.dropoff_location,
            'status': delivery.status,
            'request_time': delivery.request_time.isoformat(),
            'cost': float(delivery.cost),  # Convert Decimal to float for JSON
            'notes': delivery.notes,
            'assigned_rider_id': delivery.assigned_rider_id,
            'pickup_time': delivery.pickup_time.isoformat() if delivery.pickup_time else None,
            'dropoff_time': delivery.dropoff_time.isoformat() if delivery.dropoff_time else None,

        } for delivery in deliveries]

        # Apply route optimization
        optimized_deliveries = RouteOptimizer.optimize_route(deliveries) #get optimized list.
        optimized_deliveries_data = [{
            'delivery_id': str(delivery.delivery_id),
            'pickup_location': delivery.pickup_location,
            'dropoff_location': delivery.dropoff_location,
            'status': delivery.status,
            'request_time': delivery.request_time.isoformat(),
            'cost': float(delivery.cost),  # Convert Decimal to float for JSON
             'notes': delivery.notes,
            'assigned_rider_id': delivery.assigned_rider_id,
            'pickup_time': delivery.pickup_time.isoformat() if delivery.pickup_time else None,
            'dropoff_time': delivery.dropoff_time.isoformat() if delivery.dropoff_time else None,
        } for delivery in optimized_deliveries]

        return JsonResponse({'deliveries': optimized_deliveries_data}, status=200)



class DeliveryUpdateView(View):
    """
    View to update the status of a delivery.
    """
    def post(self, request, delivery_id):
        try:
            data = json.loads(request.body)
            new_status = data.get('status')
            if not new_status:
                return JsonResponse({'error': 'Status is required'}, status=400)

            try:
                delivery = Delivery.objects.get(delivery_id=delivery_id)
            except ObjectDoesNotExist:
                return JsonResponse({'error': 'Delivery not found'}, status=404)

            delivery.status = new_status
            delivery.save()

            # Update status in Firestore
            db.collection('deliveries').document(str(delivery.delivery_id)).update({'status': new_status})

            return JsonResponse({'message': 'Delivery status updated successfully'}, status=200)
        except json.JSONDecodeError:
            return JsonResponse({'error': 'Invalid JSON'}, status=400)
        except Exception as e:
            return JsonResponse({'error': str(e)}, status=500)

class AssignRiderView(View):
    """
    View to assign a rider to a delivery.
    """
    def post(self, request, delivery_id):
        try:
            data = json.loads(request.body)
            rider_id = data.get('rider_id')  # Firebase User ID of the rider
            if not rider_id:
                return JsonResponse({'error': 'Rider ID is required'}, status=400)

            try:
                delivery = Delivery.objects.get(delivery_id=delivery_id)
            except ObjectDoesNotExist:
                return JsonResponse({'error': 'Delivery not found'}, status=404)

            #check if rider exists
            try:
                firebase_admin.auth.get_user(rider_id)
            except firebase_admin.auth.UserNotFoundError:
                return JsonResponse({'error': 'Rider does not exist'}, status=404)

            delivery.assigned_rider_id = rider_id
            delivery.save()

            # Update in Firestore
            db.collection('deliveries').document(str(delivery.delivery_id)).update({'assigned_rider_id': rider_id})

            return JsonResponse({'message': 'Rider assigned to delivery successfully'}, status=200)
        except json.JSONDecodeError:
            return JsonResponse({'error': 'Invalid JSON'}, status=400)
        except Exception as e:
            return JsonResponse({'error': str(e)}, status=500)

class UserProfileView(View):
    """
    View to get a user's profile information.  This retrieves data from our
    local database, which should contain data *in addition* to what's in Firebase.
    """
    def get(self, request, user_id):
        try:
            user = User.objects.get(user_id=user_id)
            user_data = {
                'user_id': user.user_id,
                'name': user.name,
                'email': user.email,
                'phone': user.phone,
                'address': user.address,
            }
            return JsonResponse({'user': user_data}, status=200)
        except ObjectDoesNotExist:
            return JsonResponse({'error': 'User not found'}, status=404)
        except Exception as e:
            return JsonResponse({'error': str(e)}, status=500)


# MARK: - Django URL Configuration (urls.py)
from django.urls import path
urlpatterns = [
    path('delivery/request/', DeliveryRequestView.as_view(), name='delivery_request'),
    path('deliveries/', DeliveryListView.as_view(), name='delivery_list'),
    path('delivery/update/<uuid:delivery_id>/', DeliveryUpdateView.as_view(), name='delivery_update'),
    path('delivery/assign_rider/<uuid:delivery_id>/', AssignRiderView.as_view(), name='delivery_assign_rider'),
    path('user/<str:user_id>/profile/', UserProfileView.as_view(), name='user_profile'), #get user profile
]

# MARK: - Django settings.py (Add these settings)
# Add REST framework to your installed apps:
INSTALLED_APPS = [
    ...
    'rest_framework',
    'django.contrib.gis', #add this
    ...
]

# Database configuration (use a database that supports GeoDjango, like PostgreSQL with PostGIS)
DATABASES = {
    'default': {
        'ENGINE': 'django.contrib.gis.db.backends.postgresql',  # Or 'django.contrib.gis.db.backends.mysql', etc.
        'NAME': 'your_database_name',
        'USER': 'your_database_user',
        'PASSWORD': 'your_database_password',
        'HOST': 'localhost',
        'PORT': '5432',
    }
}
#If you are not using postgis, use this
# DATABASES = {
#     'default': {
#         'ENGINE': 'django.db.backends.sqlite3',
#         'NAME': 'mydatabase.sqlite3',
#     }
# }

# ROOT_URLCONF = 'your_project_name.urls' #Add this

# SECURITY WARNING: keep the secret key used in production secret!
SECRET_KEY = 'your_secret_key' #Add this

# SECURITY WARNING: don't run with debug turned on in production!
DEBUG = True #Add this

ALLOWED_HOSTS = ['*'] #Add this

# Application definition
# Application definition
INSTALLED_APPS = [
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
    'rest_framework',
    'django.contrib.gis',
    'your_app_name',  # Make sure to include your app
]
MIDDLEWARE = [
    'django.middleware.security.SecurityMiddleware',
    'django.contrib.sessions.middleware.SessionMiddleware',
    'django.middleware.common.CommonMiddleware',
    'django.middleware.csrf.CsrfViewMiddleware',
    'django.contrib.auth.middleware.AuthenticationMiddleware',
    'django.contrib.messages.middleware.MessageMiddleware',
    'django.middleware.clickjacking.XFrameOptionsMiddleware',
]
TEMPLATES = [
    {
        'BACKEND': 'django.template.backends.django.DjangoTemplates',
        'DIRS': [],
        'APP_DIRS': True,
        'OPTIONS': {
            'context_processors': [
                'django.template.context_processors.debug',
                'django.template.context_processors.request',
                'django.contrib.auth.context_processors.auth',
                'django.contrib.messages.context_processors.messages',
            ],
        },
    },
]
# WSGI_APPLICATION = 'your_project_name.wsgi.application' #Add this
STATIC_URL = '/static/'
DEFAULT_AUTO_FIELD = 'django.db.models.BigAutoField'